### Long Context Handling Capability

This notebook illustrates how you can use `capabilities` to give any `Conversable` agent an ability to handle long contexts.

In [1]:
## Uncomment to install pyautogen if you don't have it already
#! pip install pyautogen

In [2]:
import autogen
from autogen.agentchat.contrib.capabilities import context_handling

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-3.5-turbo"],
    },
)

In [3]:
assistant = autogen.AssistantAgent(
    "assistant",
    llm_config={
        "config_list": config_list,
    },
)
manage_chat_history = context_handling.TransformChatHistory(max_tokens_per_message=50, max_messages=10, max_tokens=1000)
manage_chat_history.add_to_agent(assistant)

user_proxy = autogen.UserProxyAgent(
    "user_proxy",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: "TERMINATE" in x.get("content", ""),
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
    max_consecutive_auto_reply=10,
)

user_proxy.initiate_chat(assistant, message="plot and save a graph of x^2 from -10 to 10")

user_proxy (to assistant):

plot and save a graph of x^2 from -10 to 10

--------------------------------------------------------------------------------
assistant (to user_proxy):

Here's a Python code snippet to plot and save a graph of x^2 from -10 to 10:

```python
import matplotlib.pyplot as plt
import numpy as np

# Generate x values from -10 to 10
x = np.linspace(-10, 10, 100)

# Evaluate y values using x^2
y = x**2

# Plot the graph
plt.plot(x, y)

# Set labels and title
plt.xlabel('x')
plt.ylabel('y')
plt.title('Graph of y = x^2')

# Save the graph as an image file (e.g., PNG)
plt.savefig('x_squared_graph.png')

# Show the graph
plt.show()
```

Please make sure to have the `matplotlib` library installed in your Python environment. After executing the code, the graph will be saved as "x_squared_graph.png" in the current directory. You can change the filename if desired.

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BL

This capability is especially useful if you expect the agent histories to become exceptionally large and exceed the context length offered by your LLM.

For example, in the example below we show what happens if this capability is available vs its not.

In [4]:
assistant = autogen.AssistantAgent(
    "assistant",
    llm_config={
        "config_list": config_list,
    },
)
# suppose this capability is not available
# manage_chat_history = context_handling.TransformChatHistory(max_tokens_per_message=50, max_messages=10, max_tokens=1000)
# manage_chat_history.add_to_agent(assistant)

user_proxy = autogen.UserProxyAgent(
    "user_proxy",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: "TERMINATE" in x.get("content", ""),
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
    max_consecutive_auto_reply=2,
)

# suppose the chat history is large
# Create a very long chat history that is bound to cause a crash
# for gpt 3.5
long_history = []
for i in range(1000):
    assitant_msg = {"role": "assistant", "content": "test " * 1000}
    user_msg = {"role": "user", "content": ""}

    assistant.send(assitant_msg, user_proxy, request_reply=False, silent=True)
    user_proxy.send(user_msg, assistant, request_reply=False, silent=True)

user_proxy.initiate_chat(assistant, message="plot and save a graph of x^2 from -10 to 10", clear_history=False)

user_proxy (to assistant):

plot and save a graph of x^2 from -10 to 10

--------------------------------------------------------------------------------


BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens. However, your messages resulted in 1009487 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [5]:
assistant = autogen.AssistantAgent(
    "assistant",
    llm_config={
        "config_list": config_list,
    },
)
# suppose this capability is not available
manage_chat_history = context_handling.TransformChatHistory(max_tokens_per_message=50, max_messages=10, max_tokens=1000)
manage_chat_history.add_to_agent(assistant)

user_proxy = autogen.UserProxyAgent(
    "user_proxy",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: "TERMINATE" in x.get("content", ""),
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
    max_consecutive_auto_reply=2,
)

# suppose the chat history is large
# Create a very long chat history that is bound to cause a crash
# for gpt 3.5
long_history = []
for i in range(1000):
    assitant_msg = {"role": "assistant", "content": "test " * 1000}
    user_msg = {"role": "user", "content": ""}

    assistant.send(assitant_msg, user_proxy, request_reply=False, silent=True)
    user_proxy.send(user_msg, assistant, request_reply=False, silent=True)

user_proxy.initiate_chat(assistant, message="plot and save a graph of x^2 from -10 to 10", clear_history=False)

user_proxy (to assistant):

plot and save a graph of x^2 from -10 to 10

--------------------------------------------------------------------------------
Truncated 1991 messages.
Truncated 49800 tokens.
assistant (to user_proxy):

Here is the Python code to plot and save a graph of x^2 from -10 to 10:

```python
import matplotlib.pyplot as plt
import numpy as np

# Generate values for x from -10 to 10
x = np.linspace(-10, 10, 100)

# Calculate y values for x^2
y = x ** 2

# Plot the graph
plt.plot(x, y)
plt.xlabel('x')
plt.ylabel('y = x^2')
plt.title('Graph of y = x^2')

# Save the graph as a PNG file
plt.savefig('graph.png')

# Close the plot
plt.close()

print('Graph saved as graph.png')
```

Please make sure you have the `matplotlib` library installed before running this code. You can install it by running `pip install matplotlib` in your terminal.

After executing the code, a graph of y = x^2 will be saved as `graph.png` in your current working directory.

Let me know if you need a